In [16]:
import tensorflow as tf
import numpy as np
import datetime
import nibabel as nib

import UNET, Pix2Pix, utilities as util, datainputfn as inputfn

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
path = "/home/francesco/UQ/Job/QSM-GAN/data/shapes_shape64_ex512_2019_05_01/"

X, Y = [], []
for i in range(1, 512):
    X.append(nib.load(path + "traintrain1-size64-ex512_" + str(i) + "_forward_tfrange.nii").get_data())
    Y.append(nib.load(path + "traintrain1-size64-ex512_" + str(i) + "_ground_truth_tfrange.nii").get_data())

X = np.array(X)
Y = np.array(Y)

X = np.expand_dims(X, axis=-1)
Y = np.expand_dims(Y, axis=-1)

print(X.shape)
print(Y.shape)

print(Y.max())
print(Y.min())

OSError: [Errno 24] Too many open files

In [18]:
last_path = "shapes_shape16_ex5_2019_03_26"
path = "/home/francesco/UQ/Job/deepQSM/data/" + last_path

currenttime = datetime.datetime.now()

checkpointName = str(currenttime.year) + "-" + str(currenttime.month) + "-" + \
                    str(currenttime.day) + "_" + str(currenttime.hour) + str(currenttime.minute) + \
                    "_" + last_path
print(checkpointName)

2019-5-7_1532_shapes_shape16_ex5_2019_03_26


In [19]:
tf.reset_default_graph()

input_shape = (16, 16, 16, 1)


#train_data_filename = util.generate_file_list(file_path=path + "/train/", p_shape=input_shape)
#eval_data_filename = util.generate_file_list(file_path=path + "/train/", p_shape=input_shape)

#train_input_fn = inputfn.data_input_fn(train_data_filename, p_shape=input_shape,
#                                       batch=1, nepochs=1, shuffle=True)

#eval_input_fn = inputfn.data_input_fn(eval_data_filename, p_shape=input_shape, 
#                                      batch=5, nepochs=1, shuffle=False)

# Unpack tensors
#train_data = train_input_fn()
#val_data = eval_input_fn()

# Construct a suitable iterator (and ops) for switching between the datasets
#iterator = tf.data.Iterator.from_structure(train_data[2].output_types, train_data[2].output_shapes)
#X_tensor, Y_tensor = iterator.get_next()

#training_init_op = iterator.make_initializer(train_data[2])
#validation_init_op = iterator.make_initializer(val_data[2])

In [20]:
X_tensor = tf.placeholder(tf.float32, shape=[None, input_shape[0], input_shape[1], input_shape[2], input_shape[3]], name='X')
Y_tensor = tf.placeholder(tf.float32, shape=[None, input_shape[0], input_shape[1], input_shape[2], input_shape[3]], name='Y')

with tf.variable_scope("generator"):
    Y_generated = Pix2Pix.getGenerator(X_tensor)

with tf.name_scope("real_discriminator"):
    with tf.variable_scope("discriminator"):
        D_logits_real = Pix2Pix.getDiscriminator(X_tensor, Y_tensor)
        
with tf.name_scope("fake_discriminator"):
    with tf.variable_scope("discriminator", reuse=True):
        D_logits_fake = Pix2Pix.getDiscriminator(X_tensor, Y_generated)

accuracy = tf.reduce_mean(tf.abs(Y_generated - Y_tensor))

Generator
Tensor("X:0", shape=(?, 16, 16, 16, 1), dtype=float32)
Tensor("generator/encoder_1/conv3d/BiasAdd:0", shape=(?, 8, 8, 8, 64), dtype=float32)
Tensor("generator/encoder_2/batch_normalization_v1/batchnorm/add_1:0", shape=(?, 4, 4, 4, 128), dtype=float32)
Tensor("generator/encoder_3/batch_normalization_v1_1/batchnorm/add_1:0", shape=(?, 2, 2, 2, 256), dtype=float32)
Tensor("generator/encoder_4/batch_normalization_v1_2/batchnorm/add_1:0", shape=(?, 1, 1, 1, 512), dtype=float32)
Tensor("generator/decoder_4/batch_normalization_v1_3/batchnorm/add_1:0", shape=(?, 2, 2, 2, 256), dtype=float32)
Tensor("generator/decoder_3/batch_normalization_v1_4/batchnorm/add_1:0", shape=(?, 4, 4, 4, 128), dtype=float32)
Tensor("generator/decoder_2/batch_normalization_v1_5/batchnorm/add_1:0", shape=(?, 8, 8, 8, 64), dtype=float32)
Tensor("generator/decoder_1/conv3d_transpose/Reshape_1:0", shape=(?, 16, 16, 16, 1), dtype=float32)

Discriminator
Tensor("real_discriminator/discriminator/concat:0", shape=(

In [21]:
'''tf.reset_default_graph()

X_tensor = tf.placeholder(tf.float32, shape=[None, X.shape[1], X.shape[2], X.shape[3], X.shape[4]], name='X')
Y_tensor = tf.placeholder(tf.float32, shape=[None, Y.shape[1], Y.shape[2], Y.shape[3], Y.shape[4]], name='Y')

# Generator
Y_generated = UNET.getNetwork(X_tensor)
print(Y_generated)

# Discriminator
D_logits_real = network.getDiscriminator(tf.concat([X_tensor, Y_tensor], -1))
D_logits_fake = network.getDiscriminator(tf.concat([X_tensor, Y_generated], -1), reuse=True)
print(D_logits_real)
print(D_logits_fake)'''

"tf.reset_default_graph()\n\nX_tensor = tf.placeholder(tf.float32, shape=[None, X.shape[1], X.shape[2], X.shape[3], X.shape[4]], name='X')\nY_tensor = tf.placeholder(tf.float32, shape=[None, Y.shape[1], Y.shape[2], Y.shape[3], Y.shape[4]], name='Y')\n\n# Generator\nY_generated = UNET.getNetwork(X_tensor)\nprint(Y_generated)\n\n# Discriminator\nD_logits_real = network.getDiscriminator(tf.concat([X_tensor, Y_tensor], -1))\nD_logits_fake = network.getDiscriminator(tf.concat([X_tensor, Y_generated], -1), reuse=True)\nprint(D_logits_real)\nprint(D_logits_fake)"

In [22]:
lr = 0.0002
batch_size = 1
min_val = 1000

In [23]:
EPS = 1e-12

def discriminatorLoss(D_real, D_fake):
   
    discrim_loss = tf.reduce_mean(-(tf.log(D_real + EPS) + tf.log(1 - D_fake + EPS)))
    
    return discrim_loss

def generatorLoss(D_fake, G_output, target, weight=100.0):
    gen_loss_GAN = tf.reduce_mean(-tf.log(D_fake + EPS))
    gen_loss_L1 = tf.reduce_mean(tf.abs(target -  G_output))
    gen_loss = gen_loss_GAN * 1.0 + gen_loss_L1 * weight

    return gen_loss, gen_loss_GAN, gen_loss_L1

# Losses
D_loss = discriminatorLoss(D_logits_real, D_logits_fake)
G_loss, G_gan, G_L1 = generatorLoss(D_logits_fake, Y_generated, Y_tensor)

# Optimizers
'''D_var = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='discriminator')
D_optimizer = tf.train.AdamOptimizer(learning_rate=lr).minimize(D_loss, var_list=D_var)

G_var = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='generator')
G_optimizer = tf.train.AdamOptimizer(learning_rate=lr).minimize(G_loss, var_list=G_var)'''

with tf.name_scope('optimize'):
    discrim_tvars = [var for var in tf.trainable_variables() if var.name.startswith("discriminator")]
    discrim_optim = tf.train.AdamOptimizer(lr, 0.5)
    discrim_grads_and_vars = discrim_optim.compute_gradients(D_loss, var_list=discrim_tvars)
    discrim_train = discrim_optim.apply_gradients(discrim_grads_and_vars)

    with tf.control_dependencies([discrim_train]):
        gen_tvars = [var for var in tf.trainable_variables() if var.name.startswith("generator")]
        gen_optim = tf.train.AdamOptimizer(lr, 0.5)
        gen_grads_and_vars = gen_optim.compute_gradients(G_loss, var_list=gen_tvars)
        gen_train = gen_optim.apply_gradients(gen_grads_and_vars)

train_op = gen_train

In [8]:
# Create a list of summaries
tf.summary.scalar("D_loss", D_loss)
tf.summary.scalar("G_L1", G_L1)
tf.summary.scalar("G_gan", G_gan)

tf.summary.image('input', X_tensor["x"][:, :, :, 8], max_outputs=1)
tf.summary.image('output', Y_generated[:, :, :, 8], max_outputs=1)
tf.summary.image('ground_truth', Y_tensor[:, :, :, 8], max_outputs=1)

# Merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

logs_path = '../checkpoints'
# tensorboard --logdir=.

with tf.Session() as sess:
    
    #saver = tf.train.Saver()
    sess.run(tf.global_variables_initializer())
    
    # op to write logs to Tensorboard
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

    total_G_loss, total_D_loss = [], [] 
    i = 0
    for _ in range(2500):
        sess.run(training_init_op)
        while True:
            try:
                _, summary = sess.run([train_op, merged_summary_op])
                summary_writer.add_summary(summary, i)
                i += 1
            except tf.errors.OutOfRangeError:
                break  

        '''sess.run(validation_init_op)
        while True:
            try:
                acc = sess.run(accuracy)
                D_loss_val = np.mean(np.array(total_D_loss))
                G_loss_val = np.mean(np.array(total_G_loss))                
                
                if acc < min_val:
                    min_val = acc
                    save_path = saver.save(sess, "../checkpoints/model.ckpt")
                    print("SAVED " + str(D_loss_val) + " " + str(G_loss_val) + " " + str(min_val))
                else:
                    print(str(D_loss_val) + " " + str(G_loss_val) + " " + str(acc))
                total_G_loss, total_D_loss = [], []
            except tf.errors.OutOfRangeError:
                break'''

KeyboardInterrupt: 

In [24]:
with tf.Session() as sess:

    saver = tf.train.Saver()
    sess.run(tf.global_variables_initializer())

    for n in range(3000):
        
        total_G_loss, total_D_loss = [], []
        for j in range(0, len(X), batch_size):
        
            # Optimize discriminator
            _, D_loss_val = sess.run([discrim_train, D_loss], feed_dict={X_tensor: X[j:j+batch_size],
                                                                         Y_tensor: Y[j:j+batch_size]})
            # Optimizer generator
            _, G_loss_val = sess.run([gen_train, G_gan], feed_dict={X_tensor: X[j:j+batch_size],
                                                                    Y_tensor: Y[j:j+batch_size]})
            

            _, G_loss_val, D_loss_val = sess.run([train_op, G_gan, D_loss], feed_dict={X_tensor: X[j:j+batch_size],
                                                          Y_tensor: Y[j:j+batch_size]})
            total_G_loss.append(G_loss_val)
            total_D_loss.append(D_loss_val)

        # Get L1 accuracy
        acc = sess.run(accuracy, feed_dict={X_tensor: X, Y_tensor: Y})
        D_loss_val = np.mean(np.array(total_D_loss))
        G_loss_val = np.mean(np.array(total_G_loss))

        if acc < min_val:
            min_val = acc
            save_path = saver.save(sess, "../checkpoints/model.ckpt")
            print("SAVED " + str(D_loss_val) + " " + str(G_loss_val) + " " + str(min_val))
        else:
            print(str(D_loss_val) + " " + str(G_loss_val) + " " + str(acc))

SAVED 1.2140636 0.71795607 0.11446164
SAVED 0.6377224 1.1591829 0.11142416
SAVED 0.083343044 3.0479674 0.09904699
SAVED 7.161979 0.0021000835 0.07922016
SAVED 1.9013597 0.5479966 0.0636034
SAVED 1.3167478 0.64377326 0.053653352
1.1992065 1.3468575 0.06675715
SAVED 1.2771196 1.5743518 0.039716803
SAVED 1.3976945 0.5848418 0.03758063
SAVED 1.4949958 0.55569 0.036703456
SAVED 1.4916941 0.52330196 0.031400643
SAVED 1.4615399 0.5632348 0.02924775
1.5452657 0.49866873 0.03332451
SAVED 1.5287694 0.48324335 0.025599211
SAVED 1.4524856 0.56540453 0.024826296
1.6053352 0.45351803 0.026701389
SAVED 1.5690409 0.4709841 0.022635177
1.5352315 0.5090013 0.023055699
SAVED 1.5847158 0.47281525 0.02134616
SAVED 1.513006 0.53926504 0.019244526
1.5014393 0.5671524 0.024472747
SAVED 1.6027713 0.4799173 0.018346619
SAVED 1.4435191 0.6266315 0.017799791
1.5739243 0.52684104 0.020009182
SAVED 1.5753345 0.54816425 0.016950518
1.459739 0.6097373 0.017909542
SAVED 1.5983981 0.495414 0.014941087
SAVED 1.3617852 0

KeyboardInterrupt: 

In [ ]:
with tf.Session() as sess:
    saver = tf.train.Saver()
    sess.run(tf.global_variables_initializer())
    sess.run(training_init_op)
    saver.restore(sess, "../checkpoints/model.ckpt") 

    img_gen, loss_val, Y, X = sess.run([Y_generated, accuracy, Y_tensor, X_tensor])
    
    print(loss_val)
    print(np.mean(np.abs(img_gen - Y)))

In [ ]:
print(img_gen[0,12,5,:,0])
print(Y[0,12,5,:,0])

In [ ]:
import matplotlib.pyplot as plt

img_ind = 0
z = 8

data = np.squeeze(img_gen[img_ind,:,:,z], axis=2)
print(data.shape)
print(X["x"].shape)

plt.imshow(data, cmap="gray")
plt.show()

plt.imshow(np.squeeze(X["x"][img_ind,:,:,z], axis=2), cmap="gray")
plt.show()

plt.imshow(np.squeeze(Y[img_ind,:,:,z], axis=2), cmap="gray")
plt.show()